In [16]:
import numpy as np


from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt #Para graficar
from skimage.io import imshow #Para graficar las imagenes
import tensorflow as tf


import os
import glob
##from PIL import Image
import cv2

#import pyheif

import matplotlib.pyplot as plt

In [17]:
#root_path = "C:/MaestriaDocs/DMA - Proyecto/caras_grupo4/"
root_path = "../"

directorio_personas = root_path + "Caras_aumentadas"

Obtenemos de las carpetas originales las fotos

In [18]:
from PCA_funciones import matriz_fotos_desde_carpeta

image_matrix, image_person = matriz_fotos_desde_carpeta(directorio_personas)

Carpeta: Abel
Carpeta: Carlos
Carpeta: Federico G
Carpeta: Federico R
Carpeta: Florencia
Carpeta: Franco A
Carpeta: Franco S
Carpeta: Gerard
Carpeta: Gustavo
Carpeta: Joaquin
Carpeta: Juan
Carpeta: Lautaro
Carpeta: Lisandro
Carpeta: Marco
Carpeta: Matias
Carpeta: Natalia
Carpeta: Noelia
Carpeta: Paola
Carpeta: Victorio


## Aplicamos Backpropagation
Transformamos las imagenes, con la cantidad de copoenentes 

In [19]:
from sklearn.decomposition import PCA
import pickle

# Cargar los componentes PCA guardados
ruta_pca = root_path + "PCA/Componentes_pca.npy" #lo tenemos en memoria pero lo leemos nuevamente por si separamos el codigo
pca_components = np.load(ruta_pca)

# Cargar el escalador
ruta_scaler = root_path + 'PCA/scaler.pkl' #
with open(ruta_scaler, 'rb') as f:
    scaler = pickle.load(f)

# Función para transformar una imagen con un rango específico de componentes PCA
def transformar_imagen_pca(imagen, scaler, pca_components, num_componentes=50, inicio_componente=0):
    # Estandarizar la imagen
    imagen_estandarizada = scaler.transform(imagen)
    
    # Seleccionar el rango de componentes
    componentes_seleccionados = pca_components[inicio_componente:inicio_componente + num_componentes]
    
    # Realizar la proyección manualmente
    imagen_pca = np.dot(imagen_estandarizada, componentes_seleccionados.T)
    return imagen_pca


# Especificar la cantidad de componentes a utilizar y el componente de inicio
num_componentes = 60
inicio_componente = 2 #inicia a partir de la 3ta componente

# Transformar la nueva imagen utilizando los componentes especificados
imagenes_pca_reducidas = transformar_imagen_pca( image_matrix, scaler, pca_components, num_componentes, inicio_componente)

print(imagenes_pca_reducidas)

imagenes_pca_reducidas.shape
personas = np.unique(image_person)
image_person #(vector que tiene el nombre de la persona de cada foto)
personas
personas.shape
cantidad_personas = len(personas)
print(cantidad_personas) 
cantidad_fotos = len(image_person)
cantidad_fotos
# salida_lista = []  # Array vacío para almacenar las listas de 0 y 1
# cantidad_personas = len(personas) #podiramos hacer un elemento menos pero por ahora lo dejamos en el total
# # Recorrer cada elemento en "image_person"
# for imagen_persona in image_person:
#   lista_persona = [0] * cantidad_personas   # Inicializar lista con 18 ceros
  
#   # Buscar el índice del nombre de la imagen en "personas"
#  # Encontrar el índice de la persona
#   indice_persona =  np.where(personas == imagen_persona)[0]

#   # Si se encuentra la persona, actualizar la lista
#   if len(indice_persona) > 0:
#     lista_persona[indice_persona[0]] = 1
  
#   # Actualizar el elemento correspondiente en la lista con 1

  
#   # Agregar la lista a Y
#   salida_lista.append(lista_persona)

# salida_lista = np.array(salida_lista) 
# salida_lista

imagenes_pca_reducidas[0]
X=imagenes_pca_reducidas
X.shape
from sklearn.preprocessing import  LabelBinarizer
# Convertir los valores de la var categórica en números
#salida_lista = LabelBinarizer().fit_transform(image_person) #convierte los nombres (var categorica) en vextores con un 1 en la posicion del nombre
#correspondiente, tiene un renglon por foto y el vector una columna por nombre solo un 1 en el nombre de la persona de la foto


label_binarizer = LabelBinarizer()
salida_lista = label_binarizer.fit_transform(image_person)
salida_lista
salida_lista.shape
X = imagenes_pca_reducidas
Y = salida_lista

# Normalización de las entradas
"""mean = np.mean(X, axis=0)
std = np.std(X, axis=0)
X = (X - mean) / std
"""
# Convierte la lista a un arreglo NumPy
#salida_lista = nombres_personas
salida = np.array(salida_lista)
Y = salida #salida.reshape(len(X),1)

epoch_limit = 5000


# Semilla para reproducibilidad
np.random.seed(1021) #Establece la semilla para las funciones aleatorias de numpy.  
tf.random.set_seed(1021) #Establece la semilla para las funciones aleatorias de TensorFlow, asegurando que los pesos iniciales de la red neuronal y 4cualquier otra operación aleatoria en TensorFlow sean reproducibles.
from sklearn.model_selection import train_test_split

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=1021)

[[-10.28285184  -1.40383104   2.95326815 ...   0.74552249   0.07381989
   -1.58577491]
 [-13.12451281  -1.96006399  -0.03303123 ...   1.65155091  -1.12603703
   -1.68849604]
 [ -6.18538294   2.2319318    1.90897576 ...  -1.51024063   0.58782629
   -0.31806208]
 ...
 [  1.89733428  -1.51669952  -5.06479381 ...  -1.83746736  -0.22823502
   -0.44339804]
 [  1.09996716   0.55458213  -5.72674711 ...  -0.60648858  -1.89728868
   -2.34782981]
 [  2.02078097  -1.40511106  -5.50231812 ...   0.97444318  -0.16456276
   -1.88983518]]
19


## Back propagation manual

In [20]:
import numpy as np
import math

# Asumiendo que ya tienes X_train, X_test, Y_train, Y_test definidos
# Inicialización de parámetros
input_size = X_train.shape[1]  # Número de características (componentes PCA)
hidden_size = 128  # Número de neuronas en la capa oculta
output_size = 19  # Número de clases (cantidad de personas)

# Inicialización de pesos y biases
np.random.seed(1021)
W1 = np.random.randn(hidden_size, input_size) * 0.01
b1 = np.zeros((hidden_size, 1))
W2 = np.random.randn(output_size, hidden_size) * 0.01
b2 = np.zeros((output_size, 1))

# Funciones de activación
def relu(Z):
    return np.maximum(0, Z)

def softmax(Z):
    expZ = np.exp(Z - np.max(Z))
    return expZ / expZ.sum(axis=0, keepdims=True)

def relu_derivative(Z):
    return Z > 0

# Función de pérdida: entropía cruzada
def categorical_crossentropy(y_true, y_pred):
    m = y_true.shape[1]
    loss = -np.sum(y_true * np.log(y_pred)) / m
    return loss

# Forward propagation
def forward_propagation(X, W1, b1, W2, b2):
    Z1 = np.dot(W1, X.T) + b1
    A1 = relu(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

# Backpropagation
def backpropagation(X, Y, Z1, A1, Z2, A2, W1, W2, b1, b2, learning_rate):
    m = X.shape[0]

    # Derivadas de la capa de salida
    dZ2 = A2 - Y.T
    dW2 = (1/m) * np.dot(dZ2, A1.T)
    db2 = (1/m) * np.sum(dZ2, axis=1, keepdims=True)

    # Derivadas de la capa oculta
    dA1 = np.dot(W2.T, dZ2)
    dZ1 = dA1 * relu_derivative(Z1)
    dW1 = (1/m) * np.dot(dZ1, X)
    db1 = (1/m) * np.sum(dZ1, axis=1, keepdims=True)

    # Actualización de los parámetros
    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1
    W2 -= learning_rate * dW2
    b2 -= learning_rate * db2

    return W1, b1, W2, b2

# Entrenamiento del modelo
epochs = epoch_limit
learning_rate = 0.5
for epoch in range(epochs):
    # Forward propagation
    Z1, A1, Z2, A2 = forward_propagation(X_train, W1, b1, W2, b2)

    # Cálculo de la pérdida
    loss = categorical_crossentropy(Y_train.T, A2)

    # Aquí puedes imprimir el avance en cada epoch
    if epoch % 100 == 0 or epoch == epochs - 1:  # Imprime cada 100 epochs o el último epoch
        # Calcular precisión en el conjunto de entrenamiento
        predictions_train = np.argmax(A2, axis=0)
        labels_train = np.argmax(Y_train, axis=1)
        accuracy_train = np.mean(predictions_train == labels_train)
        
        print(f"Epoch {epoch}, Loss: {loss:.6f}, Training Accuracy: {accuracy_train * 100:.2f}%")
    
    # Backpropagation
    W1, b1, W2, b2 = backpropagation(X_train, Y_train, Z1, A1, Z2, A2, W1, W2, b1, b2, learning_rate)



Epoch 0, Loss: 2.943873, Training Accuracy: 6.18%
Epoch 100, Loss: 0.097149, Training Accuracy: 98.37%
Epoch 200, Loss: 0.038203, Training Accuracy: 99.66%
Epoch 300, Loss: 0.021482, Training Accuracy: 99.87%
Epoch 400, Loss: 0.014107, Training Accuracy: 99.94%
Epoch 500, Loss: 0.010201, Training Accuracy: 99.99%
Epoch 600, Loss: 0.007861, Training Accuracy: 100.00%
Epoch 700, Loss: 0.006328, Training Accuracy: 100.00%
Epoch 800, Loss: 0.005260, Training Accuracy: 100.00%
Epoch 900, Loss: 0.004477, Training Accuracy: 100.00%
Epoch 1000, Loss: 0.003882, Training Accuracy: 100.00%
Epoch 1100, Loss: 0.003415, Training Accuracy: 100.00%
Epoch 1200, Loss: 0.003041, Training Accuracy: 100.00%
Epoch 1300, Loss: 0.002735, Training Accuracy: 100.00%
Epoch 1400, Loss: 0.002481, Training Accuracy: 100.00%
Epoch 1500, Loss: 0.002267, Training Accuracy: 100.00%
Epoch 1600, Loss: 0.002084, Training Accuracy: 100.00%
Epoch 1700, Loss: 0.001926, Training Accuracy: 100.00%
Epoch 1800, Loss: 0.001789, T

In [21]:
# Evaluación del modelo en el conjunto de prueba
Z1_test, A1_test, Z2_test, A2_test = forward_propagation(X_test, W1, b1, W2, b2)
test_loss = categorical_crossentropy(Y_test.T, A2_test)
print(f"Test Loss: {test_loss}")

# Calcular precisión en el conjunto de prueba
predictions_test = np.argmax(A2_test, axis=0)
labels_test = np.argmax(Y_test, axis=1)
accuracy_test = np.mean(predictions_test == labels_test)
print(f"Test Accuracy: {accuracy_test * 100:.2f}%")


Test Loss: 0.04145104030922963
Test Accuracy: 99.41%


In [22]:
# Propagación hacia adelante en el conjunto de prueba
#Z1_test, A1_test, Z2_test, A2_test = forward_propagation(X_test, W1, b1, W2, b2)

# Iterar sobre las predicciones y mostrar el resultado para cada foto
for i in range(len(A2_test.T)):
    # Obtener la predicción para la i-ésima foto
    prediction = A2_test.T[i]
    
    # Encontrar la etiqueta predicha (la clase con probabilidad más alta)
    predicted_label = np.argmax(prediction)  # Identifica la clase que el modelo considera más probable para la imagen
    
    # Mostrar el resultado real y la predicción
    real_label = np.argmax(Y_test[i])
    predicted_probability = np.max(prediction)
    
    print(f"Foto {i+1}: Real = {real_label}, Predicción = {predicted_label}, Probabilidad = {predicted_probability:.4f}, Probabilidades = {prediction}")


Foto 1: Real = 4, Predicción = 4, Probabilidad = 1.0000, Probabilidades = [6.06389193e-09 4.99033065e-15 4.84179707e-16 1.66602408e-11
 9.99999977e-01 4.18935337e-16 9.14741975e-16 1.61852967e-15
 4.89922493e-10 1.16459034e-16 1.48204014e-13 6.61211756e-12
 2.08910778e-15 1.61717301e-17 1.98337577e-13 1.25664576e-08
 2.39051827e-09 1.16203008e-09 1.69991080e-11]
Foto 2: Real = 16, Predicción = 16, Probabilidad = 0.9874, Probabilidades = [1.24326264e-02 3.38569651e-08 2.31589668e-05 6.54416346e-06
 3.54013649e-05 1.84485672e-09 2.04906775e-09 5.84035508e-05
 4.47460687e-06 5.85848464e-12 2.05295396e-07 4.41467786e-05
 3.92371406e-06 3.46111947e-08 7.24068026e-09 1.97904601e-09
 9.87390878e-01 4.47827686e-08 1.11236199e-07]
Foto 3: Real = 2, Predicción = 2, Probabilidad = 1.0000, Probabilidades = [1.31321681e-11 2.34337240e-23 9.99999991e-01 2.47235736e-11
 8.17727669e-25 6.68895424e-23 1.98885637e-19 9.27849426e-20
 1.43782118e-13 4.54618604e-28 8.99111877e-12 5.29085094e-28
 1.42903896

In [23]:


# Obtener las predicciones (clase con mayor probabilidad)
y_predicho = np.argmax(A2_test, axis=0)  # axis=0 porque A2_test tiene la forma (clases, muestras)
y_testeo = np.argmax(Y_test, axis=1)  # axis=1 para obtener las etiquetas reales

# Identificar los errores de predicción
error_i = (y_predicho != y_testeo)

# Mostrar los índices donde ocurrieron los errores
print("Errores de predicción en los índices:", np.where(error_i)[0])
print("Total de errores:", np.sum(error_i))


Errores de predicción en los índices: [ 236  464  743  780  995 1084 1109 1287 1288 1355]
Total de errores: 10


In [24]:
import pandas as pd

In [25]:
# Convertir índices de clases a nombres de clases
y_test_names = label_binarizer.inverse_transform(A2_test.T) #reales de test
y_pred_names = label_binarizer.inverse_transform(Y_test)


# Crear DataFrame
tabla_resultados = pd.DataFrame({
    'y_predicho': y_pred_names,
    'y_testeo': y_test_names
})
tabla_resultados['Predicción correcta'] = tabla_resultados['y_predicho'] == tabla_resultados['y_testeo']
tabla_resultados

,y_predicho,y_testeo,Predicción correcta
0,Florencia,Florencia,True
1,Noelia,Noelia,True
2,Federico G,Federico G,True
3,Federico R,Federico R,True
4,Matias,Matias,True
...,...,...,...
1682,Marco,Marco,True
1683,Federico G,Federico G,True
1684,Natalia,Natalia,True
1685,Lisandro,Lisandro,True


In [26]:
tabla_resultados_T=tabla_resultados[tabla_resultados['Predicción correcta'] == True]

In [27]:
tabla_resultados_F=tabla_resultados[tabla_resultados['Predicción correcta'] == False]

In [28]:
tabla_resultados_T

,y_predicho,y_testeo,Predicción correcta
0,Florencia,Florencia,True
1,Noelia,Noelia,True
2,Federico G,Federico G,True
3,Federico R,Federico R,True
4,Matias,Matias,True
...,...,...,...
1682,Marco,Marco,True
1683,Federico G,Federico G,True
1684,Natalia,Natalia,True
1685,Lisandro,Lisandro,True


In [29]:
tabla_resultados_F

,y_predicho,y_testeo,Predicción correcta
236,Carlos,Franco S,False
464,Abel,Noelia,False
743,Noelia,Gerard,False
780,Joaquin,Lautaro,False
995,Victorio,Joaquin,False
1084,Matias,Lautaro,False
1109,Victorio,Gerard,False
1287,Franco S,Matias,False
1288,Franco S,Matias,False
1355,Noelia,Lautaro,False


## Guardar pesos

In [30]:
ruta_modelo = os.path.join(root_path , 'Back_propagation')
ruta_modelo = os.path.join(ruta_modelo , 'Modelo')

#print(ruta_modelo_3c)
# Guardar los pesos y biases en la carpeta especificada
np.save(os.path.join(ruta_modelo, "W1.npy"), W1)
np.save(os.path.join(ruta_modelo, "b1.npy"), b1)
np.save(os.path.join(ruta_modelo, "W2.npy"), W2)
np.save(os.path.join(ruta_modelo, "b2.npy"), b2)
#np.save(os.path.join(ruta_modelo, "W3.npy"), W3)
#np.save(os.path.join(ruta_modelo, "X03.npy"), X03)